# GMTRouter - Inference

This notebook demonstrates how to use a trained **GMTRouter** for inference.

## 1. Environment Setup

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)

In [ ]:
import torch
from llmrouter.models.gmtrouter import GMTRouter
from llmrouter.utils import setup_environment
import yaml

setup_environment()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 2. Load Trained Router

In [ ]:
CONFIG_PATH = "configs/model_config_train/gmtrouter.yaml"

router = GMTRouter(yaml_path=CONFIG_PATH)
print("Router loaded!")

## 3. Personalized Query Routing

In [ ]:
# Multi-turn conversation example
CONVERSATION = [
    {
        "query": "What is machine learning?",
        "user_id": "user_001",
        "session_id": "session_001",
        "turn": 1
    },
    {
        "query": "Can you give me a practical example?",
        "user_id": "user_001",
        "session_id": "session_001",
        "turn": 2
    },
    {
        "query": "How do I implement this in Python?",
        "user_id": "user_001",
        "session_id": "session_001",
        "turn": 3
    },
]

print("Multi-turn Routing Results:")
print("=" * 60)

for query in CONVERSATION:
    result = router.route_single(query)
    print(f"Turn {query['turn']}: {query['query'][:40]}...")
    print(f"   Routed to: {result['model_name']}")

## 4. Different User Preferences

In [ ]:
# Same query, different users
query_text = "Explain neural networks."

users = ["user_001", "user_002", "user_003"]

print(f"Query: {query_text}")
print("=" * 60)

for user in users:
    query = {
        "query": query_text,
        "user_id": user,
        "session_id": f"{user}_session"
    }
    result = router.route_single(query)
    print(f"{user}: Routed to {result['model_name']}")

## 5. File-Based Inference

Load queries from a file and save results.

In [ ]:
import json

# Load queries from a JSONL file
def load_queries_from_file(file_path):
    """Load queries from a JSONL file."""
    queries = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                queries.append(json.loads(line))
    return queries

# Save results to a JSONL file
def save_results_to_file(results, output_path):
    """Save routing results to a JSONL file."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(json.dumps(result, ensure_ascii=False) + '\n')
    print(f"Results saved to: {output_path}")

# Example: Load from default query file
QUERY_FILE = "data/example_data/query_data/default_query_test.jsonl"
OUTPUT_FILE = "outputs/gmtrouter_results.jsonl"

if os.path.exists(QUERY_FILE):
    # Load queries
    file_queries = load_queries_from_file(QUERY_FILE)
    print(f"Loaded {len(file_queries)} queries from: {QUERY_FILE}")
    
    # Route queries
    file_results = router.route_batch(batch=file_queries[:10])
    print(f"Routed {len(file_results)} queries")
    
    # Save results
    save_results_to_file(file_results, OUTPUT_FILE)
    
    # Show sample results
    print(f"\nSample results:")
    for i, result in enumerate(file_results[:3], 1):
        print(f"  {i}. {result.get('query', '')[:40]}... -> {result['model_name']}")
else:
    print(f"Query file not found: {QUERY_FILE}")
    print("Create a JSONL file with format: {\"query\": \"Your question\"}")

## Summary

GMTRouter provides:
- Personalized routing based on user history
- Multi-turn context awareness
- Graph-based relationship modeling